In [1]:
import xml.sax
from xml.dom.minidom import parse
import xml.dom.minidom
import re
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters
from stanfordcorenlp import StanfordCoreNLP
nlp=StanfordCoreNLP('G:\\stanford-corenlp-full-2018-10-05',lang='en')


In [2]:
#load data file
def load_strtonum_feed(filename):
    data = []
    data_str = ''
    with open(filename, 'r', encoding='UTF-8') as f:#with auto call close()
        line = f.readline()
        while line:
            #read_data = line.split('\n')
            #data_str +=read_data
            data_str +=line
            #data.append(read_data[0])
            line = f.readline()
        return data_str 

In [3]:
craft_ids_dev = load_strtonum_feed("craft-ids-dev.txt").split("\n")

In [4]:
craft_ids_train = load_strtonum_feed("craft-ids-train.txt").split("\n")

In [5]:
craft_ids_test = load_strtonum_feed("craft-ids-test.txt").split("\n")

In [6]:
#load file
def txt_strtonum_feed(filename):
    data = []
    data_str = ''
    with open(filename, 'r', encoding='UTF-8') as f:#with auto call close()
        line = f.readline()
        while line:
            #read_data = line.split('\n')
            #data_str +=read_data
            data_str +=line
            #data.append(read_data[0])
            line = f.readline()
        return data_str 

In [7]:
def handle_data(filename,ann_filename):
        
    txt = txt_strtonum_feed(filename)
    DOMTree = xml.dom.minidom.parse(ann_filename)
    collection = DOMTree.documentElement
    annotations = collection.getElementsByTagName("annotation")
    data_list = []
    for annotation in annotations:
        mention = annotation.getElementsByTagName('class')[0].getAttribute("id")

        for span in annotation.getElementsByTagName('span'):
            end = int(span.getAttribute("end")  )   
            start =int(span.getAttribute("start")) 
            data = span.childNodes[0].data
            data_list.append((start,end,mention,data))
    data_list.sort()
    
    
    # 获取开始的list
    spanStart_list=[]
    spanEnd_list=[]
    # 获取文本
    spannedText_list=[]
    pr_list=[]
    for i in data_list:
        spanStart_list.append(i[0])
        spanEnd_list.append(i[1])
        spannedText_list.append(i[3])
        pr_list.append(i[2])
        
        
    punkt_param = PunktParameters()
    abbreviation = ['i.e']
    punkt_param.abbrev_types = set(abbreviation)
    sent_tokenizer = PunktSentenceTokenizer(punkt_param)
    
    sentence_list =[]
    data = re.split("\n\n",txt)
    for i in data:
        sentence_list = sentence_list + sent_tokenizer.tokenize(i)
    data = sentence_list
    
    #####################################3
    #处理文本  new_tag_list ==
    """
    [[('ocular', 'UBERON:0000970', 5, 11),
    ('genetically', 'SO:0000704', 24, 35),
    ('mice', 'NCBITaxon:10088', 45, 49)],
    [],
    """
    sentence_list = []
    j = 0
    number = 0
    last_number = 0
    new_tag_list = []
    sub_tag_list = []
    for i in range(len(data)):
        number_ = txt[last_number:].find(data[i]) + last_number
        
        last_number= number_ + len(data[i])
        
        
        while (j < len(spanEnd_list)) and  (spanEnd_list[j] <= last_number) and (spanEnd_list[j] >= number_):
            sub_tag_list.append((spannedText_list[j],pr_list[j],int(spanStart_list[j])-number_ ,int(spanEnd_list[j])-number_))
            j+=1
        new_tag_list.append(sub_tag_list)
        sub_tag_list=[]

        
        
        
        
    #######################################
    #先将文本处理成对应位置 doc_out_put===
    """
    [[('Intraocular', 'JJ', 0, 11),
      ('pressure', 'NN', 12, 20),
      ('in', 'IN', 21, 23),
      ('genetically', 'RB', 24, 35),
      ('distinct', 'JJ', 36, 44),
      ('mice', 'NNS', 45, 49),
      (':', ':', 49, 50),
      ('an', 'DT', 51, 53),
      ('update', 'VB', 54, 60),
      ('and', 'CC', 61, 64),
      ('strain', 'VB', 65, 71),
      ('survey', 'NN', 72, 78)],
     [('Abstract', 'JJ', 0, 8)],
    """
    doc_out_put = []
    error_list = []
    mark_list = ['(']
    for i in range(len(data)):
        sub_output = []
        doc = nlp.pos_tag(data[i])
        index_start = 0
        index_end = 0

        
        for j in range(len(doc)):
            index_start = data[i][index_end:].find(doc[j][0]) + index_end
            index_end = index_start + len(doc[j][0])
            sub_output.append((doc[j][0],doc[j][1],index_start,index_end))              
        doc_out_put.append(sub_output)
    
    
    
    #######################################
    # 标注数据
    store_data = []
    for i in range(len(doc_out_put)):
        sub_store_data = []
        index_ = 0
        for j in doc_out_put[i]:
            label = 'O'
            if  new_tag_list[i]:

                #如果文本 比 标注的长， 那么让标注 滑动
                while j[2] <= new_tag_list[i][index_][2] and j[3] >= new_tag_list[i][index_][3]:
                        label = "B"
#                         label = new_tag_list[i][index_][1]
                        if index_ < len(new_tag_list[i])-1:
                            index_+=1
                        else:
                            break
                #如果文本比 标注短， 让文本滑动
                if j[2] == new_tag_list[i][index_][2] and j[3] < new_tag_list[i][index_][3]:
#                     label = new_tag_list[i][index_][1]
                    label = "B"
                elif j[2] > new_tag_list[i][index_][2] and j[3] < new_tag_list[i][index_][3]:
#                     label = new_tag_list[i][index_][1]
                    label = "B"
                elif j[2] > new_tag_list[i][index_][2] and j[3] == new_tag_list[i][index_][3]:
#                     label = new_tag_list[i][index_][1]
                    label = "B"
                    if index_ < len(new_tag_list[i])-1:
                        index_+=1    


            sub_store_data.append((j[0],j[1],label))
        store_data.append(sub_store_data) 
        
        
    return store_data

In [8]:
store_data_train = []
for i in craft_ids_train:
    filename = "data/"+i+".txt"
    ann_filename = "data/"+i+".xml"
    store_data_train += handle_data(filename,ann_filename)

In [9]:
store_data_dev = []
for i in craft_ids_dev:
    filename = "data/"+i+".txt"
    ann_filename = "data/"+i+".xml"
    store_data_dev += handle_data(filename,ann_filename)

In [10]:
store_data_test = []
for i in craft_ids_test:
    filename = "data/"+i+".txt"
    ann_filename = "data/"+i+".xml"
    store_data_test += handle_data(filename,ann_filename)

In [11]:
import pickle
fw = open('craft_ids_train.pl','wb')
# Pickle the list using the highest protocol available.
#pickle.dump(dataList, fw, -1)
# Pickle dictionary using protocol 0.
pickle.dump(store_data_train, fw)
fw.close()

In [12]:
fw = open('craft_ids_dev.pl','wb')
# Pickle the list using the highest protocol available.
#pickle.dump(dataList, fw, -1)
# Pickle dictionary using protocol 0.
pickle.dump(store_data_dev, fw)
fw.close()

In [13]:
fw = open('craft_ids_test.pl','wb')
# Pickle the list using the highest protocol available.
#pickle.dump(dataList, fw, -1)
# Pickle dictionary using protocol 0.
pickle.dump(store_data_test, fw)
fw.close()

In [46]:
import pickle
fw = open('craft_ids_train_detail.pl','wb')
# Pickle the list using the highest protocol available.
#pickle.dump(dataList, fw, -1)
# Pickle dictionary using protocol 0.
pickle.dump(store_data_train, fw)
fw.close()

In [47]:
fw = open('craft_ids_dev_detail.pl','wb')
# Pickle the list using the highest protocol available.
#pickle.dump(dataList, fw, -1)
# Pickle dictionary using protocol 0.
pickle.dump(store_data_dev, fw)
fw.close()

In [48]:
fw = open('craft_ids_test_detail.pl','wb')
# Pickle the list using the highest protocol available.
#pickle.dump(dataList, fw, -1)
# Pickle dictionary using protocol 0.
pickle.dump(store_data_test, fw)
fw.close()

In [1]:
import pickle
fr1 = open('craft_ids_train_detail.pl','rb')

fr2 = open('craft_ids_dev_detail.pl','rb')

fr3 = open('craft_ids_test_detail.pl','rb')

store_data_train = pickle.load(fr1)
store_data_dev = pickle.load(fr2)
store_data_test = pickle.load(fr3)

In [1]:
import pickle
fr1 = open('craft_ids_train.pl','rb')

fr2 = open('craft_ids_dev.pl','rb')

fr3 = open('craft_ids_test.pl','rb')

store_data_train = pickle.load(fr1)
store_data_dev = pickle.load(fr2)
store_data_test = pickle.load(fr3)

In [20]:
import nltk
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()


## try lemmatizer

In [21]:
store_data_trainx=[]
store_data_trainy=[]
for i in store_data_train:
    sub_trainx = []
    sub_trainy = []
    for j in i:
        word = lemmatizer.lemmatize(j[0])
        sub_trainx.append(word)
        sub_trainy.append(j[2])
    store_data_trainx.append(sub_trainx)
    store_data_trainy.append(sub_trainy)

In [22]:
store_data_devx=[]
store_data_devy=[]
for i in store_data_dev:
    sub_trainx = []
    sub_trainy = []
    for j in i:
        word = lemmatizer.lemmatize(j[0])
        sub_trainx.append(word)
        sub_trainy.append(j[2])
    store_data_devx.append(sub_trainx)
    store_data_devy.append(sub_trainy)

In [23]:
store_data_testx=[]
store_data_testy=[]
for i in store_data_test:
    sub_trainx = []
    sub_trainy = []
    for j in i:
        word = lemmatizer.lemmatize(j[0])
        sub_trainx.append(word)
        sub_trainy.append(j[2])
    store_data_testx.append(sub_trainx)
    store_data_testy.append(sub_trainy)

## normal 

In [2]:
store_data_trainx=[]
store_data_trainy=[]
for i in store_data_train:
    sub_trainx = []
    sub_trainy = []
    for j in i:
        word = j[0]+j[1]
#         sub_trainx.append(word)
        sub_trainx.append(j[0])
        sub_trainy.append(j[2])
    store_data_trainx.append(sub_trainx)
    store_data_trainy.append(sub_trainy)

In [3]:
store_data_devx=[]
store_data_devy=[]
for i in store_data_dev:
    sub_trainx = []
    sub_trainy = []
    for j in i:
        word = j[0]+j[1]
#         sub_trainx.append(word)
        sub_trainx.append(j[0])
        sub_trainy.append(j[2])
    store_data_devx.append(sub_trainx)
    store_data_devy.append(sub_trainy)

In [4]:
store_data_testx=[]
store_data_testy=[]
for i in store_data_test:
    sub_trainx = []
    sub_trainy = []
    for j in i:
        word = j[0]+j[1]
#         sub_trainx.append(word)
        sub_trainx.append(j[0])
        sub_trainy.append(j[2])
    store_data_testx.append(sub_trainx)
    store_data_testy.append(sub_trainy)

In [10]:
# 打开一个文件
fo = open("store_data_trainx.txt", "w",encoding='gb18030')
for i, sub_data_i in enumerate(store_data_trainx):
    for j,sub_data_j  in enumerate(sub_data_i):
        string = sub_data_j+"\t"+store_data_trainy[i][j]+"\n"
        fo.write(string)
    fo.write("\n")
        
# 关闭打开的文件
fo.close()

In [11]:
# 打开一个文件
fo = open("train_dev.tsv", "w",encoding='gb18030')
for i, sub_data_i in enumerate(store_data_devx):
    for j,sub_data_j  in enumerate(sub_data_i):
        string = sub_data_j+"\t"+store_data_devy[i][j]+"\n"
        fo.write(string)
    fo.write("\n")
        
# 关闭打开的文件
fo.close()

In [12]:
# 打开一个文件
fo = open("test.tsv", "w",encoding='gb18030')
for i, sub_data_i in enumerate(store_data_testx):
    for j,sub_data_j  in enumerate(sub_data_i):
        string = sub_data_j+"\t"+store_data_testy[i][j]+"\n"
        fo.write(string)
    fo.write("\n")
        
# 关闭打开的文件
fo.close()

In [10]:
store_data_testy[0]

['O', 'O', 'O', 'O', 'O', 'B', 'B', 'O', 'O', 'B', 'B', 'O', 'O', 'B', 'O']

## split :   to  normalization ##

In [23]:
store_data_trainx=[]
store_data_trainy=[]
for i in store_data_train:
    sub_trainx = []
    sub_trainy = []
    for j in i:
#         word = j[2].split(":")[0]
        sub_trainx.append(j[0])
        sub_trainy.append(word)
    store_data_trainx.append(sub_trainx)
    store_data_trainy.append(sub_trainy)

In [24]:
store_data_devx=[]
store_data_devy=[]
for i in store_data_dev:
    sub_trainx = []
    sub_trainy = []
    for j in i:
#         word = j[2].split(":")[0]
        sub_trainx.append(j[0])
        sub_trainy.append(word)
    store_data_devx.append(sub_trainx)
    store_data_devy.append(sub_trainy)

In [25]:
store_data_testx=[]
store_data_testy=[]
for i in store_data_test:
    sub_trainx = []
    sub_trainy = []
    for j in i:
#         word = j[2].split(":")[0]
        sub_trainx.append(j[0])
        sub_trainy.append(word)
    store_data_testx.append(sub_trainx)
    store_data_testy.append(sub_trainy)

In [11]:
import kashgari
kashgari.config.use_cudnn_cell = True
from kashgari.tasks.labeling import BiLSTM_Model

In [16]:
model = BiLSTM_Model()
#valid_x, valid_y = trainx[:250], trainy[:250]
model.fit(store_data_trainx, store_data_trainy, store_data_devx, store_data_devy,epochs=40)

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 50)]              0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 50, 100)           1172900   
_________________________________________________________________
layer_blstm (Bidirectional)  (None, 50, 256)           235520    
_________________________________________________________________
layer_dropout (Dropout)      (None, 50, 256)           0         
_________________________________________________________________
layer_time_distributed (Time (None, 50, 3)             771       
_________________________________________________________________
activation_2 (Activation)    (None, 50, 3)             0         
Total params: 1,409,191
Trainable params: 1,409,191
Non-trainable params: 0
_________________________________________________

In [17]:
# 验证模型，此方法将打印出详细的验证报告
model.evaluate(store_data_testx, store_data_testy)

           precision    recall  f1-score   support

        B     0.8119    0.7171    0.7616     31293

micro avg     0.8119    0.7171    0.7615     31293
macro avg     0.8119    0.7171    0.7616     31293



'           precision    recall  f1-score   support\n\n        B     0.8119    0.7171    0.7616     31293\n\nmicro avg     0.8119    0.7171    0.7615     31293\nmacro avg     0.8119    0.7171    0.7616     31293\n'

In [ ]:
micro avg     0.8690    0.6592    0.7497     31293
macro avg     0.8692    0.6592    0.7498     31293

In [9]:
model.save("BiLSTM10")

In [12]:
model.predict([store_data_testx[0]])

[['O', 'O', 'O', 'O', 'O', 'B', 'B', 'O', 'O', 'B', 'B', 'O', 'O', 'B', 'O']]

In [10]:
store_data_testx[0]

['Complex',
 'trait',
 'analysis',
 'of',
 'the',
 'mouse',
 'striatum',
 ':',
 'independent',
 'QTLs',
 'modulate',
 'volume',
 'and',
 'neuron',
 'number']

In [21]:
model = BiLSTM_Model()
#valid_x, valid_y = trainx[:250], trainy[:250]
model.fit(store_data_trainx, store_data_trainy, store_data_devx, store_data_devy,epochs=50)

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 50)]              0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 50, 100)           1101800   
_________________________________________________________________
layer_blstm (Bidirectional)  (None, 50, 256)           235520    
_________________________________________________________________
layer_dropout (Dropout)      (None, 50, 256)           0         
_________________________________________________________________
layer_time_distributed (Time (None, 50, 3)             771       
_________________________________________________________________
activation_1 (Activation)    (None, 50, 3)             0         
Total params: 1,338,091
Trainable params: 1,338,091
Non-trainable params: 0
_________________________________________________

In [22]:
# 验证模型，此方法将打印出详细的验证报告
model.evaluate(store_data_testx, store_data_testy)

           precision    recall  f1-score   support

        B     0.8081    0.7272    0.7655     31293

micro avg     0.8080    0.7272    0.7655     31293
macro avg     0.8081    0.7272    0.7655     31293



'           precision    recall  f1-score   support\n\n        B     0.8081    0.7272    0.7655     31293\n\nmicro avg     0.8080    0.7272    0.7655     31293\nmacro avg     0.8081    0.7272    0.7655     31293\n'

In [23]:
model = BiLSTM_Model()
#valid_x, valid_y = trainx[:250], trainy[:250]
model.fit(store_data_trainx, store_data_trainy, store_data_devx, store_data_devy,epochs=100)

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 50)]              0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 50, 100)           1101800   
_________________________________________________________________
layer_blstm (Bidirectional)  (None, 50, 256)           235520    
_________________________________________________________________
layer_dropout (Dropout)      (None, 50, 256)           0         
_________________________________________________________________
layer_time_distributed (Time (None, 50, 3)             771       
_________________________________________________________________
activation_2 (Activation)    (None, 50, 3)             0         
Total params: 1,338,091
Trainable params: 1,338,091
Non-trainable params: 0
_________________________________________________

314/314 [==============================] - 3s 11ms/step - loss: 6.9616e-04 - acc: 0.9998 - val_loss: 0.3107 - val_acc: 0.9674
Epoch 68/100
314/314 [==============================] - 3s 10ms/step - loss: 4.2218e-04 - acc: 0.9999 - val_loss: 0.3153 - val_acc: 0.9664
Epoch 69/100
314/314 [==============================] - 3s 11ms/step - loss: 2.5751e-04 - acc: 0.9999 - val_loss: 0.3441 - val_acc: 0.9654
Epoch 70/100
314/314 [==============================] - 3s 11ms/step - loss: 1.7904e-04 - acc: 1.0000 - val_loss: 0.3439 - val_acc: 0.9657
Epoch 71/100
314/314 [==============================] - 3s 11ms/step - loss: 2.0908e-04 - acc: 0.9999 - val_loss: 0.3448 - val_acc: 0.9652
Epoch 72/100
314/314 [==============================] - 3s 11ms/step - loss: 3.7110e-04 - acc: 0.9999 - val_loss: 0.3449 - val_acc: 0.9661
Epoch 73/100
314/314 [==============================] - 3s 11ms/step - loss: 2.8508e-04 - acc: 0.9999 - val_loss: 0.3357 - val_acc: 0.9661
Epoch 74/100
314/314 [==================

In [24]:
# 验证模型，此方法将打印出详细的验证报告
model.evaluate(store_data_testx, store_data_testy)

           precision    recall  f1-score   support

        B     0.7998    0.7345    0.7658     31293

micro avg     0.7998    0.7345    0.7658     31293
macro avg     0.7998    0.7345    0.7658     31293



'           precision    recall  f1-score   support\n\n        B     0.7998    0.7345    0.7658     31293\n\nmicro avg     0.7998    0.7345    0.7658     31293\nmacro avg     0.7998    0.7345    0.7658     31293\n'

In [ ]:
micro avg     0.8084    0.7150    0.7588     31293
macro avg     0.8088    0.7150    0.7590     31293

micro avg     0.8080    0.7272    0.7655     31293
macro avg     0.8081    0.7272    0.7655     31293


micro avg     0.7998    0.7345    0.7658     31293
macro avg     0.7998    0.7345    0.7658     31293


In [ ]:
# 10

micro avg     0.7338    0.4433    0.5527     27140
macro avg     0.4554    0.4433    0.4385     27140


# 30
micro avg     0.7192    0.5065    0.5944     27140
macro avg     0.5328    0.5065    0.5094     27140




# 50 
 micro avg     0.6940    0.5132    0.5900     27140
macro avg     0.5288    0.5132    0.5104     27140

In [ ]:



#using 
#10
           micro avg     0.7569    0.5656    0.6474     26737
           macro avg     0.7240    0.5656    0.6186     26737

        
#30
           micro avg     0.6966    0.5906    0.6392     26737
           macro avg     0.6777    0.5906    0.6262     26737
        


#50 
           micro avg     0.6947    0.6079    0.6484     26737
           macro avg     0.6787    0.6079    0.6384     26737

In [9]:
model = BiLSTM_Model()
#valid_x, valid_y = trainx[:250], trainy[:250]
model.fit(store_data_trainx, store_data_trainy, store_data_devx, store_data_devy,epochs=50)

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 50)]              0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 50, 100)           1101800   
_________________________________________________________________
layer_blstm (Bidirectional)  (None, 50, 256)           235520    
_________________________________________________________________
layer_dropout (Dropout)      (None, 50, 256)           0         
_________________________________________________________________
layer_time_distributed (Time (None, 50, 3)             771       
_________________________________________________________________
activation_1 (Activation)    (None, 50, 3)             0         
Total params: 1,338,091
Trainable params: 1,338,091
Non-trainable params: 0
_________________________________________________

In [8]:
# 验证模型，此方法将打印出详细的验证报告
model.evaluate(store_data_testx, store_data_testy)

           precision    recall  f1-score   support

        B     0.8290    0.7096    0.7647     31293

micro avg     0.8290    0.7096    0.7647     31293
macro avg     0.8290    0.7096    0.7647     31293



'           precision    recall  f1-score   support\n\n        B     0.8290    0.7096    0.7647     31293\n\nmicro avg     0.8290    0.7096    0.7647     31293\nmacro avg     0.8290    0.7096    0.7647     31293\n'

In [6]:
from kashgari.embeddings import BERTEmbedding


In [8]:
bert_embed = BERTEmbedding('wwm_cased_L-24_H-1024_A-16',task=kashgari.LABELING,sequence_length=200)

In [7]:
bert_embed = BERTEmbedding('biobert_large',task=kashgari.LABELING,sequence_length="auto")

In [34]:
model = BiLSTM_Model(bert_embed)
#valid_x, valid_y = trainx[:250], trainy[:250]
model.fit(store_data_trainx, store_data_trainy, store_data_devx, store_data_devy,epochs=10)

Model: "model_17"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 50)]         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 50)]         0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 50, 1024), ( 60411904    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 50, 1024)     2048        Input-Segment[0][0]              
___________________________________________________________________________________________

Epoch 1/10
314/314 [==============================] - 232s 740ms/step - loss: 0.0906 - acc: 0.9591 - val_loss: 0.0785 - val_acc: 0.9617
Epoch 2/10
314/314 [==============================] - 218s 693ms/step - loss: 0.0691 - acc: 0.9680 - val_loss: 0.0732 - val_acc: 0.9652
Epoch 3/10
314/314 [==============================] - 219s 696ms/step - loss: 0.0648 - acc: 0.9700 - val_loss: 0.0720 - val_acc: 0.9647
Epoch 4/10
314/314 [==============================] - 217s 691ms/step - loss: 0.0617 - acc: 0.9714 - val_loss: 0.0741 - val_acc: 0.9637
Epoch 5/10
314/314 [==============================] - 216s 689ms/step - loss: 0.0598 - acc: 0.9723 - val_loss: 0.0705 - val_acc: 0.9664
Epoch 6/10
314/314 [==============================] - 221s 702ms/step - loss: 0.0584 - acc: 0.9728 - val_loss: 0.0714 - val_acc: 0.9655
Epoch 7/10
314/314 [==============================] - 220s 702ms/step - loss: 0.0566 - acc: 0.9735 - val_loss: 0.0717 - val_acc: 0.9657
Epoch 8/10
314/314 [============================

In [35]:
# 验证模型，此方法将打印出详细的验证报告  10
model.evaluate(store_data_testx, store_data_testy)

           precision    recall  f1-score   support

        B     0.7542    0.7699    0.7620     31241

micro avg     0.7542    0.7699    0.7620     31241
macro avg     0.7542    0.7699    0.7620     31241



'           precision    recall  f1-score   support\n\n        B     0.7542    0.7699    0.7620     31241\n\nmicro avg     0.7542    0.7699    0.7620     31241\nmacro avg     0.7542    0.7699    0.7620     31241\n'

In [16]:
import kashgari
kashgari.config.use_cudnn_cell = True
from kashgari.tasks.labeling import BiLSTM_CRF_Model   

In [46]:
from kashgari.embeddings import BERTEmbedding
bert_embed = BERTEmbedding('biobert_large',task=kashgari.LABELING,sequence_length="auto")

In [39]:
model = BiLSTM_CRF_Model(bert_embed)
#valid_x, valid_y = trainx[:250], trainy[:250]
model.fit(store_data_trainx, store_data_trainy, store_data_devx, store_data_devy,epochs=40)

Model: "model_22"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 50)]         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 50)]         0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 50, 1024), ( 60411904    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 50, 1024)     2048        Input-Segment[0][0]              
___________________________________________________________________________________________

Epoch 1/40
314/314 [==============================] - 266s 848ms/step - loss: 4.8221 - accuracy: 0.9590 - val_loss: 27.0064 - val_accuracy: 0.8234
Epoch 2/40
314/314 [==============================] - 258s 822ms/step - loss: 3.4838 - accuracy: 0.9690 - val_loss: 25.5063 - val_accuracy: 0.8226
Epoch 3/40
314/314 [==============================] - 263s 837ms/step - loss: 3.1052 - accuracy: 0.9720 - val_loss: 24.0899 - val_accuracy: 0.8246
Epoch 4/40
314/314 [==============================] - 262s 835ms/step - loss: 2.8604 - accuracy: 0.9738 - val_loss: 23.7578 - val_accuracy: 0.7220
Epoch 5/40
314/314 [==============================] - 263s 836ms/step - loss: 2.6929 - accuracy: 0.9751 - val_loss: 23.3369 - val_accuracy: 0.6403
Epoch 6/40
314/314 [==============================] - 264s 839ms/step - loss: 2.5645 - accuracy: 0.9762 - val_loss: 23.0908 - val_accuracy: 0.6512
Epoch 7/40
314/314 [==============================] - 263s 837ms/step - loss: 2.4726 - accuracy: 0.9769 - val_loss: 22

In [40]:
# 验证模型，此方法将打印出详细的验证报告  10
model.evaluate(store_data_testx, store_data_testy)

           precision    recall  f1-score   support

        B     0.7792    0.6781    0.7252     31241

micro avg     0.7791    0.6781    0.7251     31241
macro avg     0.7792    0.6781    0.7252     31241



'           precision    recall  f1-score   support\n\n        B     0.7792    0.6781    0.7252     31241\n\nmicro avg     0.7791    0.6781    0.7251     31241\nmacro avg     0.7792    0.6781    0.7252     31241\n'

In [ ]:
model.save("mymodel")

In [17]:
model = BiLSTM_CRF_Model()
#valid_x, valid_y = trainx[:250], trainy[:250]
model.fit(store_data_trainx, store_data_trainy, store_data_devx, store_data_devy,epochs=3)

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 50)]              0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 50, 100)           1101800   
_________________________________________________________________
layer_blstm (Bidirectional)  (None, 50, 256)           235520    
_________________________________________________________________
layer_dense (Dense)          (None, 50, 64)            16448     
_________________________________________________________________
layer_crf_dense (Dense)      (None, 50, 3)             195       
_________________________________________________________________
layer_crf (CRF)              (None, 50, 3)             9         
Total params: 1,353,972
Trainable params: 1,353,972
Non-trainable params: 0
_________________________________________________

In [18]:
# 验证模型，此方法将打印出详细的验证报告 BiLSTM_CRF_Model  3
model.evaluate(store_data_testx, store_data_testy)

           precision    recall  f1-score   support

        B     0.8804    0.6536    0.7503     31293

micro avg     0.8802    0.6536    0.7502     31293
macro avg     0.8804    0.6536    0.7503     31293



'           precision    recall  f1-score   support\n\n        B     0.8804    0.6536    0.7503     31293\n\nmicro avg     0.8802    0.6536    0.7502     31293\nmacro avg     0.8804    0.6536    0.7503     31293\n'

In [ ]:

#3
micro avg     0.8883    0.6406    0.7444     31293
macro avg     0.8887    0.6406    0.7445     31293

In [12]:
model = BiLSTM_CRF_Model(bert_embed)
#valid_x, valid_y = trainx[:250], trainy[:250]
model.fit(store_data_trainx, store_data_trainy, store_data_devx, store_data_devy,epochs=10)

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 50)]         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 50)]         0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 50, 1024), ( 60411904    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 50, 1024)     2048        Input-Segment[0][0]              
____________________________________________________________________________________________

Epoch 1/10
314/314 [==============================] - 268s 855ms/step - loss: 4.6245 - accuracy: 0.9572 - val_loss: 37.4303 - val_accuracy: 0.6751
Epoch 2/10
314/314 [==============================] - 260s 829ms/step - loss: 3.2577 - accuracy: 0.9686 - val_loss: 37.2014 - val_accuracy: 0.6743
Epoch 3/10
314/314 [==============================] - 263s 837ms/step - loss: 2.9952 - accuracy: 0.9711 - val_loss: 37.0037 - val_accuracy: 0.6760
Epoch 4/10
314/314 [==============================] - 261s 832ms/step - loss: 2.8334 - accuracy: 0.9727 - val_loss: 36.8267 - val_accuracy: 0.6844
Epoch 5/10
314/314 [==============================] - 261s 832ms/step - loss: 2.6948 - accuracy: 0.9741 - val_loss: 36.6618 - val_accuracy: 0.6858
Epoch 6/10
314/314 [==============================] - 261s 833ms/step - loss: 2.5778 - accuracy: 0.9752 - val_loss: 36.7249 - val_accuracy: 0.6856
Epoch 7/10
314/314 [==============================] - 262s 833ms/step - loss: 2.5017 - accuracy: 0.9759 - val_loss: 36

In [13]:
# 验证模型，此方法将打印出详细的验证报告 BiLSTM_CRF_Model(bert_embed)  3
model.evaluate(store_data_testx, store_data_testy)

           precision    recall  f1-score   support

        B     0.7583    0.7285    0.7431     31241

micro avg     0.7582    0.7285    0.7430     31241
macro avg     0.7583    0.7285    0.7431     31241



'           precision    recall  f1-score   support\n\n        B     0.7583    0.7285    0.7431     31241\n\nmicro avg     0.7582    0.7285    0.7430     31241\nmacro avg     0.7583    0.7285    0.7431     31241\n'

In [ ]:
# 10  bert

micro avg     0.7251    0.7072    0.7161     31793
macro avg     0.7251    0.7072    0.7161     31793


# biobert 10   30 sqence
micro avg     0.7827    0.7144    0.7470     27732
macro avg     0.7827    0.7144    0.7470     27732

#biobert 10  auto   95% length of sqence
micro avg     0.7542    0.7699    0.7620     31241
macro avg     0.7542    0.7699    0.7620     31241

#biobert 40  auto   95% length of sqence   crf
micro avg     0.7791    0.6781    0.7251     31241
macro avg     0.7792    0.6781    0.7252     31241

micro avg     0.8068    0.7286    0.7657     31293
macro avg     0.8068    0.7286    0.7657     31293



#biobert 3  auto   95% length of sqence   crf  bert
micro avg     0.7259    0.7796    0.7518     31241
macro avg     0.7260    0.7796    0.7518     31241

10
micro avg     0.7582    0.7285    0.7430     31241
macro avg     0.7583    0.7285    0.7431     31241

In [12]:
model = BiLSTM_Model(bert_embed)
#valid_x, valid_y = trainx[:250], trainy[:250]
model.fit(store_data_trainx, store_data_trainy, store_data_devx, store_data_devy,epochs=30)

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 100)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 100)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 100, 1024),  29691904    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 100, 1024)    2048        Input-Segment[0][0]              
____________________________________________________________________________________________

Total params: 336,435,971
Trainable params: 4,328,195
Non-trainable params: 332,107,776
__________________________________________________________________________________________________
Epoch 1/30
314/314 [==============================] - 446s 1s/step - loss: 0.0592 - acc: 0.9734 - val_loss: 0.0456 - val_acc: 0.9766
Epoch 2/30
314/314 [==============================] - 437s 1s/step - loss: 0.0410 - acc: 0.9803 - val_loss: 0.0424 - val_acc: 0.9787
Epoch 3/30
314/314 [==============================] - 439s 1s/step - loss: 0.0383 - acc: 0.9815 - val_loss: 0.0400 - val_acc: 0.9797
Epoch 4/30
314/314 [==============================] - 438s 1s/step - loss: 0.0365 - acc: 0.9824 - val_loss: 0.0396 - val_acc: 0.9800
Epoch 5/30
314/314 [==============================] - 438s 1s/step - loss: 0.0353 - acc: 0.9831 - val_loss: 0.0401 - val_acc: 0.9803
Epoch 6/30
314/314 [==============================] - 453s 1s/step - loss: 0.0344 - acc: 0.9834 - val_loss: 0.0410 - val_acc: 0.9793
Epoch 7/30
314/

In [14]:
# 验证模型，此方法将打印出详细的验证报告  30
model.evaluate(store_data_testx, store_data_testy)

           precision    recall  f1-score   support

        B     0.7072    0.7339    0.7203     31752

micro avg     0.7072    0.7339    0.7203     31752
macro avg     0.7072    0.7339    0.7203     31752



'           precision    recall  f1-score   support\n\n        B     0.7072    0.7339    0.7203     31752\n\nmicro avg     0.7072    0.7339    0.7203     31752\nmacro avg     0.7072    0.7339    0.7203     31752\n'

In [11]:
# 验证模型，此方法将打印出详细的验证报告
model.evaluate(store_data_testx, store_data_testy)

           precision    recall  f1-score   support

        B     0.7152    0.7262    0.7207     31752

micro avg     0.7152    0.7262    0.7207     31752
macro avg     0.7152    0.7262    0.7207     31752



'           precision    recall  f1-score   support\n\n        B     0.7152    0.7262    0.7207     31752\n\nmicro avg     0.7152    0.7262    0.7207     31752\nmacro avg     0.7152    0.7262    0.7207     31752\n'

In [9]:
# 验证模型，此方法将打印出详细的验证报告     50
model.evaluate(store_data_testx, store_data_testy)

           precision    recall  f1-score   support

        B     0.7316    0.6727    0.7009     31752

micro avg     0.7316    0.6727    0.7009     31752
macro avg     0.7316    0.6727    0.7009     31752



'           precision    recall  f1-score   support\n\n        B     0.7316    0.6727    0.7009     31752\n\nmicro avg     0.7316    0.6727    0.7009     31752\nmacro avg     0.7316    0.6727    0.7009     31752\n'

In [9]:
from kashgari.embeddings import BERTEmbedding
from kashgari.tasks.labeling import BiLSTM_CRF_Model
#bert_embedding = BERTEmbedding('wwm_cased_L-24_H-1024_A-16', sequence_length=100)

In [12]:
bert_embed = BERTEmbedding('wwm_cased_L-24_H-1024_A-16',task=kashgari.LABELING,sequence_length=100)

In [7]:
model = BiLSTM_Model()
#valid_x, valid_y = trainx[:250], trainy[:250]
model.fit(store_data_trainx, store_data_trainy, store_data_devx, store_data_devy,epochs=10)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 50)]              0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 50, 100)           1101800   
_________________________________________________________________
layer_blstm (Bidirectional)  (None, 50, 256)           235520    
_________________________________________________________________
layer_dropout (Dropout)      (None, 50, 256)           0         
_________________________________________________________________
layer_time_distributed (Time (None, 50, 3)             771       
_________________________________________________________________
activation (Activation)      (None, 50, 3)             0         
Total params: 1,338,091
Trainable params: 1,338,091
Non-trainable params: 0
_________________________________________________

In [8]:
# 验证模型，此方法将打印出详细的验证报告
model.evaluate(store_data_testx, store_data_testy)

           precision    recall  f1-score   support

        B     0.7911    0.7561    0.7732     31293

micro avg     0.7910    0.7561    0.7731     31293
macro avg     0.7911    0.7561    0.7732     31293



'           precision    recall  f1-score   support\n\n        B     0.7911    0.7561    0.7732     31293\n\nmicro avg     0.7910    0.7561    0.7731     31293\nmacro avg     0.7911    0.7561    0.7732     31293\n'

## after lemma word

In [15]:
model = BiLSTM_Model()
#valid_x, valid_y = trainx[:250], trainy[:250]
model.fit(store_data_trainx, store_data_trainy, store_data_devx, store_data_devy,epochs=10)

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 50)]              0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 50, 100)           1034900   
_________________________________________________________________
layer_blstm (Bidirectional)  (None, 50, 256)           235520    
_________________________________________________________________
layer_dropout (Dropout)      (None, 50, 256)           0         
_________________________________________________________________
layer_time_distributed (Time (None, 50, 3)             771       
_________________________________________________________________
activation_1 (Activation)    (None, 50, 3)             0         
Total params: 1,271,191
Trainable params: 1,271,191
Non-trainable params: 0
_________________________________________________

In [16]:
# 验证模型，此方法将打印出详细的验证报告
model.evaluate(store_data_testx, store_data_testy)

           precision    recall  f1-score   support

        B     0.8009    0.7189    0.7577     31293

micro avg     0.8007    0.7189    0.7576     31293
macro avg     0.8009    0.7189    0.7577     31293



'           precision    recall  f1-score   support\n\n        B     0.8009    0.7189    0.7577     31293\n\nmicro avg     0.8007    0.7189    0.7576     31293\nmacro avg     0.8009    0.7189    0.7577     31293\n'

In [ ]:
from bert_serving.client import BertClient
